In [ ]:
import json
import os
import numpy as np
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import MobileNetV2, ResNet50, EfficientNetB0
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import tensorflow as tf
import pandas as pd

# Define models and image sizes
models = ["CNN", "ResNet50", "MobileNetV2", "EfficientNetB0"]
image_sizes = [128, 256, 512]
base_path = "/kaggle/input/traffic-prediction-models-plotting-value/traffic_prediction_models_Plotting_Value"
test_dir = '/kaggle/input/dhaka-traffic-classification-4-levels/Dhaka City Traffic Classification Dataset - 4-Level Congestion Analysis/test'

# Initialize the model loading functions
def load_model_for_inference(model_name, size, num_classes):
    if model_name == "ResNet50":
        model = ResNet50(input_shape=(size, size, 3), include_top=False, weights='imagenet')
        model = tf.keras.Sequential([model,
                                     tf.keras.layers.GlobalAveragePooling2D(),
                                     tf.keras.layers.Dense(num_classes, activation='softmax')])
    elif model_name == "MobileNetV2":
        model = MobileNetV2(input_shape=(size, size, 3), include_top=False, weights='imagenet')
        model = tf.keras.Sequential([model,
                                     tf.keras.layers.GlobalAveragePooling2D(),
                                     tf.keras.layers.Dense(num_classes, activation='softmax')])
    elif model_name == "EfficientNetB0":
        model = EfficientNetB0(input_shape=(size, size, 3), include_top=False, weights='imagenet')
        model = tf.keras.Sequential([model,
                                     tf.keras.layers.GlobalAveragePooling2D(),
                                     tf.keras.layers.Dense(num_classes, activation='softmax')])
    else:
        # Custom CNN model, allow for different input sizes
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(size, size, 3)),  # Specify the input shape here
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
            tf.keras.layers.MaxPooling2D((2, 2)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(num_classes, activation='softmax')  # Adjust for number of classes
        ])
    return model

# Load the actual test data from the directory
def load_test_data(test_dir, img_size):
    image_paths = []
    labels = []
label_map = {}

    # Get class names from subfolders in the test directory
class_names = sorted(os.listdir(test_dir))
    for idx, class_name in enumerate(class_names):
        label_map[idx] = class_name  # Map class index to class name
        class_folder = os.path.join(test_dir, class_name)

        # For each image in the class folder
            for img_file in os.listdir(class_folder):
if img_file.endswith('.jpg') or img_file.endswith('.png'):
                image_paths.append(os.path.join(class_folder, img_file))
                labels.append(idx)  # Store the class index
    
    # Load and preprocess images
    images = []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=(img_size, img_size))
        img_array = image.img_to_array(img)
                img_array = preprocess_input(img_array)  # Preprocess according to model requirements
        images.append(img_array)
    
    # Convert lists to arrays
    images = np.array(images)
    labels = np.array(labels)

return images, labels, class_names

# Directory to save the classification reports
save_dir = '/kaggle/working/classification_reports'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# Iterate through models and image sizes, and save classification report to JSON
for model_name in models:
    for size in image_sizes:
        file_path = f"{base_path}/{model_name}_Results_{size}.json"

        # Check if the file exists
        if os.path.exists(file_path):
            # Load the saved JSON data
            with open(file_path, 'r') as f:
                data = json.load(f)

            # Extract class names (from JSON) and true test labels
            class_names = data.get("class_names", [])
            num_classes = len(class_names)  # Adjust number of classes based on the JSON

            # Load test data (images and labels)
            X_test, y_test, label_map = load_test_data(test_dir, size)

            # Load the model
            model = load_model_for_inference(model_name, size, num_classes)
            
            # Make predictions on the test data
            predictions = model.predict(X_test)
            y_pred_classes = np.argmax(predictions, axis=1)
            
            # Generate classification report with zero_division set to 1
            report = classification_report(y_test, y_pred_classes,                                          target_names=class_names, output_dict=True, zero_division=1)

            # Save the report as JSON
            report_file = os.path.join(save_dir, f"{model_name}_classification_report_{size}.json")
            with open(report_file, 'w') as f:
                json.dump(report, f, indent=4)

            # Convert the report to a pandas DataFrame for better table visualization
            report_df = pd.DataFrame(report).transpose()

            # Print the classification report as a table
            print(f"Classification report for {model_name} with image size {size}:")
            print(report_df)
            print("\n" + "="*50 + "\n")

        else:
            print(f"File {file_path} does not exist.")


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
Classification report for CNN with image size 128:
                  precision    recall  f1-score     support
heavy traffic      0.220238  0.909836  0.354633  122.000000
light traffic      0.187500  0.027027  0.047244  111.000000
moderate traffic   0.000000  0.000000  0.000000  131.000000
no traffic         0.522727  0.114428  0.187755  201.000000
accuracy           0.242478  0.242478  0.242478    0.242478
macro avg          0.232616  0.262823  0.147408  565.000000
weighted avg       0.270354  0.242478  0.152651  565.000000


18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Classification report for CNN with image size 256:
                  precision    recall  f1-score     support
heavy traffic      0.204301  0.155738  0.176744  122.000000
light traffic      0.219048  0.207207  0.212963  111.000000
moderate traffic   0.194444  0.213740  0.203636  131.000000
no traffic         0.363229  0.402985  0.382075  201.000000
accuracy           0.267257  0.26725

<ipython-input-11-64998b46e53e>:25: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(input_shape=(size, size, 3), include_top=False, weights='imagenet')


18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 150ms/step
Classification report for MobileNetV2 with image size 256:
                  precision    recall  f1-score     support
heavy traffic      0.400000  0.081967  0.136054  122.000000
light traffic      0.207547  0.297297  0.244444  111.000000
moderate traffic   0.240678  0.541985  0.333333  131.000000
no traffic         0.418605  0.179104  0.250871  201.000000
accuracy           0.265487  0.265487  0.265487    0.265487
macro avg          0.316707  0.275088  0.241176  565.000000
weighted avg       0.331869  0.265487  0.243936  565.000000




<ipython-input-11-64998b46e53e>:25: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNetV2(input_shape=(size, size, 3), include_top=False, weights='imagenet')


18/18 ━━━━━━━━━━━━━━━━━━━━ 6s 196ms/step
Classification report for MobileNetV2 with image size 512:
                  precision    recall  f1-score     support
heavy traffic      0.227488  0.786885  0.352941  122.000000
light traffic      0.000000  0.000000  0.000000  111.000000
moderate traffic   0.272727  0.022901  0.042254  131.000000
no traffic         0.423077  0.273632  0.332326  201.000000
accuracy           0.272566  0.272566  0.272566    0.272566
macro avg          0.230823  0.270854  0.181880  565.000000
weighted avg       0.262866  0.272566  0.204233  565.000000


18/18 ━━━━━━━━━━━━━━━━━━━━ 9s 240ms/step
Classification report for EfficientNetB0 with image size 128:
                  precision   recall  f1-score    support
heavy traffic      1.000000  0.00000  0.000000  122.00000
light traffic      0.196460  1.00000  0.328402  111.00000
moderate traffic   1.000000  0.00000  0.000000  131.00000
no traffic         1.000000  0.00000  0.000000  201.00000
accuracy           0.1964